In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate

from torchvision.models import resnet18
from torchvision import transforms
from torchvision.datasets import CIFAR10, UCF101

from pytorchvideo.models import x3d

import os
import pickle

In [ ]:
class Args:
    def __init__(self):
        self.metadata_path = '/dataset/'
        self.frames_per_clip = 8
        self.root = '/dataset/UCF-101/'
        self.annotation_path = '/dataset/ucfTrainTestlist/'
        self.frames_per_clip = 8
        self.step_between_clips = 8
        self.model = 'X3D'
        self.batch_size = 1
        self.num_workers = 1

args = Args()

In [ ]:
# https://www.kaggle.com/pevogam/starter-ucf101-with-pytorch
transform = transforms.Compose([
    # scale in [0, 1] of type float
    transforms.Lambda(lambda x: x / 255.),
    # reshape into (T, C, H, W) for easier convolutions
    transforms.Lambda(lambda x: x.permute(0, 3, 1, 2)),
    # rescale to the most common size
    # transforms.Lambda(
    #     lambda x: nn.functional.interpolate(x, (240, 320))),
])

In [ ]:
def remove_audio_collate(batch):
    # https://www.kaggle.com/pevogam/starter-ucf101-with-pytorch
    '''
    remove audio channel because
    not all of UCF101 vidoes have audio channel
    '''
    video_only_batch = []
    print('batch len', len(video_only_batch))
    for video, audio, label in batch:
        print('video/audio/label type',
            type(video), type(audio), type(label))
        video_only_batch.append((video, label))
    return default_collate(video_only_batch)

# custom_collate = remove_audio_collate
custom_collate = default_collate


In [ ]:
metadata_filename = os.path.join(
    args.metadata_path,
    'UCF101metadata_fpc{}_sbc{}.pickle'.format(args.frames_per_clip,
                                                args.step_between_clips))
with open(metadata_filename, 'rb') as f:
    metadata = pickle.load(f)

In [6]:
train_set = UCF101(root=args.root,
                    annotation_path=args.annotation_path,
                    frames_per_clip=args.frames_per_clip,
                    step_between_clips=args.step_between_clips,
                    fold=1,
                    train=True,
                    transform=transform,
                    _precomputed_metadata=metadata)
val_set = UCF101(root=args.root,
                    annotation_path=args.annotation_path,
                    frames_per_clip=args.frames_per_clip,
                    step_between_clips=args.step_between_clips,
                    fold=1,
                    train=False,
                    transform=transform,
                    _precomputed_metadata=metadata)
n_classes = 101

In [7]:
train_loader = DataLoader(train_set,
                            batch_size=args.batch_size,
                            shuffle=True,
                            drop_last=True,
                            collate_fn=custom_collate,
                            num_workers=args.num_workers)
val_loader = DataLoader(val_set,
                        batch_size=args.batch_size,
                        shuffle=False,
                        drop_last=True,
                        collate_fn=custom_collate,
                        num_workers=args.num_workers)


In [8]:
len(train_loader)

219119

In [9]:
train_loader_it = iter(train_loader)

In [33]:
batch = next(train_loader_it)
batch

[tensor([[[[[0.8863, 0.8863, 0.8863,  ..., 0.8745, 0.8745, 0.8745],
            [0.8863, 0.8863, 0.8863,  ..., 0.8824, 0.8824, 0.8824],
            [0.8902, 0.8902, 0.8902,  ..., 0.8824, 0.8824, 0.8824],
            ...,
            [0.2706, 0.2627, 0.2863,  ..., 0.2353, 0.2353, 0.2353],
            [0.2706, 0.2627, 0.2863,  ..., 0.2157, 0.2157, 0.2157],
            [0.2706, 0.2627, 0.2863,  ..., 0.2196, 0.2196, 0.2196]],
 
           [[0.8902, 0.8902, 0.8902,  ..., 0.8784, 0.8784, 0.8784],
            [0.8902, 0.8902, 0.8902,  ..., 0.8863, 0.8863, 0.8863],
            [0.8941, 0.8941, 0.8941,  ..., 0.8863, 0.8863, 0.8863],
            ...,
            [0.4118, 0.4039, 0.4275,  ..., 0.4000, 0.4000, 0.4000],
            [0.4118, 0.4039, 0.4275,  ..., 0.3804, 0.3804, 0.3804],
            [0.4118, 0.4039, 0.4275,  ..., 0.3843, 0.3843, 0.3843]],
 
           [[0.9020, 0.9020, 0.9020,  ..., 0.8902, 0.8902, 0.8902],
            [0.9020, 0.9020, 0.9020,  ..., 0.8980, 0.8980, 0.8980],
        

In [ ]:
len(batch)

In [ ]:
for i in batch:
    print(i.shape)

In [34]:
model = x3d.create_x3d()

In [69]:
model

   (act_b): Swish()
            (conv_c): Conv3d(108, 48, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
            (norm_c): BatchNorm3d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (activation): ReLU()
        )
      )
    )
    (3): ResStage(
      (res_blocks): ModuleList(
        (0): ResBlock(
          (branch1_conv): Conv3d(48, 96, kernel_size=(1, 1, 1), stride=(1, 2, 2), bias=False)
          (branch1_norm): BatchNorm3d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (branch2): BottleneckBlock(
            (conv_a): Conv3d(48, 216, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
            (norm_a): BatchNorm3d(216, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act_a): ReLU()
            (conv_b): Conv3d(216, 216, kernel_size=(3, 3, 3), stride=(1, 2, 2), padding=(1, 1, 1), groups=216, bias=False)
            (norm_b): Sequential(
              (0): BatchNorm3

In [64]:
model._modules

: Conv3d(96, 216, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
                        (norm_a): BatchNorm3d(216, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                        (act_a): ReLU()
                        (conv_b): Conv3d(216, 216, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), groups=216, bias=False)
                        (norm_b): Sequential(
                          (0): BatchNorm3d(216, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                          (1): Identity()
                        )
                        (act_b): Swish()
                        (conv_c): Conv3d(216, 96, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
                        (norm_c): BatchNorm3d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                      )
                      (activation): ReLU()
                    )
                    (4): ResBlock(
                      (branch2)

In [80]:
for i, m in enumerate(model.modules()):
    print(i, m._get_name())

0 Net
1 ModuleList
2 ResNetBasicStem
3 Conv2plus1d
4 Conv3d
5 Conv3d
6 BatchNorm3d
7 ReLU
8 ResStage
9 ModuleList
10 ResBlock
11 Conv3d
12 BottleneckBlock
13 Conv3d
14 BatchNorm3d
15 ReLU
16 Conv3d
17 Sequential
18 BatchNorm3d
19 SqueezeExcitation
20 Sequential
21 Conv3d
22 ReLU
23 Conv3d
24 Sigmoid
25 Swish
26 Conv3d
27 BatchNorm3d
28 ReLU
29 ResBlock
30 BottleneckBlock
31 Conv3d
32 BatchNorm3d
33 ReLU
34 Conv3d
35 Sequential
36 BatchNorm3d
37 Identity
38 Swish
39 Conv3d
40 BatchNorm3d
41 ReLU
42 ResBlock
43 BottleneckBlock
44 Conv3d
45 BatchNorm3d
46 ReLU
47 Conv3d
48 Sequential
49 BatchNorm3d
50 SqueezeExcitation
51 Sequential
52 Conv3d
53 ReLU
54 Conv3d
55 Sigmoid
56 Swish
57 Conv3d
58 BatchNorm3d
59 ReLU
60 ResStage
61 ModuleList
62 ResBlock
63 Conv3d
64 BatchNorm3d
65 BottleneckBlock
66 Conv3d
67 BatchNorm3d
68 ReLU
69 Conv3d
70 Sequential
71 BatchNorm3d
72 SqueezeExcitation
73 Sequential
74 Conv3d
75 ReLU
76 Conv3d
77 Sigmoid
78 Swish
79 Conv3d
80 BatchNorm3d
81 ReLU
82 ResBlock

In [84]:
for i, m in enumerate(model.named_modules()):
    print(i, m[0])

0 
1 blocks
2 blocks.0
3 blocks.0.conv
4 blocks.0.conv.conv_t
5 blocks.0.conv.conv_xy
6 blocks.0.norm
7 blocks.0.activation
8 blocks.1
9 blocks.1.res_blocks
10 blocks.1.res_blocks.0
11 blocks.1.res_blocks.0.branch1_conv
12 blocks.1.res_blocks.0.branch2
13 blocks.1.res_blocks.0.branch2.conv_a
14 blocks.1.res_blocks.0.branch2.norm_a
15 blocks.1.res_blocks.0.branch2.act_a
16 blocks.1.res_blocks.0.branch2.conv_b
17 blocks.1.res_blocks.0.branch2.norm_b
18 blocks.1.res_blocks.0.branch2.norm_b.0
19 blocks.1.res_blocks.0.branch2.norm_b.1
20 blocks.1.res_blocks.0.branch2.norm_b.1.block
21 blocks.1.res_blocks.0.branch2.norm_b.1.block.0
22 blocks.1.res_blocks.0.branch2.norm_b.1.block.1
23 blocks.1.res_blocks.0.branch2.norm_b.1.block.2
24 blocks.1.res_blocks.0.branch2.norm_b.1.block.3
25 blocks.1.res_blocks.0.branch2.act_b
26 blocks.1.res_blocks.0.branch2.conv_c
27 blocks.1.res_blocks.0.branch2.norm_c
28 blocks.1.res_blocks.0.activation
29 blocks.1.res_blocks.1
30 blocks.1.res_blocks.1.branch2
31 